In [ ]:
# 1. 꼬인 라이브러리들 완전히 뿌리뽑기
!pip uninstall -y unsloth unsloth_zoo torchao peft transformers trl accelerate bitsandbytes

# 2. 가장 안정적인 '표준 조합' 강제 설치 (2025년 기준 가장 에러 없는 조합)
# 핵심: peft를 조금 낮은 안정 버전(0.13.2)으로 고정하여 내부 경로 충돌을 방지합니다.
!pip install --no-cache-dir \
    torch==2.5.1+cu124 \
    torchvision==0.20.1+cu124 \
    torchaudio==2.5.1+cu124 \
    --index-url https://download.pytorch.org/whl/cu124

!pip install --no-cache-dir \
    transformers==4.46.3 \
    peft==0.13.2 \
    trl==0.12.1 \
    accelerate==1.1.1 \
    bitsandbytes==0.44.1 \
    datasets==3.1.0 \
    python-dotenv psutil

print("\n✅ 표준 안정 버전 정착 완료! 지금 즉시 [Kernel] -> [Restart Kernel] 하세요.")

🛠️ pip 복구를 시도합니다...
Looking in links: /tmp/tmp9xm699_j
  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
✅ pip 복구 성공!
🚀 의존성 설치를 다시 시작합니다...
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-z9tzocp1/unsloth_65659e4f3c6741d280c2d64f62db9ee4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-z9tzocp1/unsloth_65659e4f3c6741d280c2d64f62db9ee4
  Resolved https://github.com/unslothai/unsloth.git to commit 06daf28c8b79782375bb7e17a830b11266407bc9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.12.9-py3-none-any.whl size=382465 sha256=fe42fb688d03887c078a444ab213a32d14e88a166054850a50df

In [ ]:
import torch
import transformers
import peft
import trl

print(f"🔹 Transformers: {transformers.__version__}") # 4.46.3 확인
print(f"🔹 PEFT: {peft.__version__}")                # 0.13.2 확인
print("🚀 [성공] 이제 모든 표준 라이브러리가 정상적으로 로드됩니다!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


AttributeError: module 'torch._subclasses.fake_tensor' has no attribute 'UnsupportedMutationAliasingException'

In [ ]:
import os
import gc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset

# ==========================================
# 1. 환경 및 경로 설정
# ==========================================
MODEL_ID = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"
HF_TOKEN = 'YOUR_HF_TOKEN' # TODO INSERT
HF_REPO_ID = 'walker0625/k-lingo-finetuning'
LEVELS = ["level1", "level2", "level3"] 

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# ==========================================
# 2. 레벨별 순차 학습 루프
# ==========================================
for level in LEVELS:
    print(f"\n\n{'='*20} 🎯 {level} 학습 시작 (500개 데이터 대응) {'='*20}")
    
    DATA_PATH = f"data/{level}.jsonl"
    if not os.path.exists(DATA_PATH):
        print(f"⚠️ {DATA_PATH} 파일이 없어 건너띔.")
        continue

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
        token=HF_TOKEN
    )
    model = prepare_model_for_kbit_training(model)

    peft_config = LoraConfig(
        r=16, 
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, peft_config)

    dataset = load_dataset("json", data_files=DATA_PATH, split="train")

    def formatting_prompts_func(example):
        text = (
            f"<|im_start|>system\n{example['instruction']}<|im_end|>\n"
            f"<|im_start|>user\n{example['input']}<|im_end|>\n"
            f"<|im_start|>thought\n{example['thought']}<|im_end|>\n"
            f"<|im_start|>call\n{example['tool_call']}<|im_end|>\n"
            f"<|im_start|>observation\n{example['observation']}<|im_end|>\n"
            f"<|im_start|>assistant\n{example['output']}<|im_end|>"
        )
        tokenized = tokenizer(text, truncation=True, max_length=1024, padding=False)
        return {
            "input_ids": tokenized["input_ids"], 
            "attention_mask": tokenized["attention_mask"], 
            "labels": tokenized["input_ids"].copy()
        }

    dataset = dataset.map(formatting_prompts_func, batched=False, remove_columns=dataset.column_names)

    # [수정된 부분] 500개 데이터 / (BS 1 * Acc 8) = 약 63 steps per epoch.
    # 3 Epoch 학습 시 총 188 steps가 적당합니다.
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        max_seq_length=1024,
        args=TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=8,
            max_steps=188,               # 500개 데이터 기준 3 Epoch
            learning_rate=2e-4,
            bf16=True,
            logging_steps=5,
            optim="adamw_8bit",          # 메모리 효율을 위해 8bit optimizer 사용
            output_dir=f"outputs_{level}",
            report_to="none",
            save_strategy="no"
        ),
    )

    trainer.train()

    print(f"☁️ {level} 어댑터 업로드 중 (revision='{level}')")
    model.push_to_hub(HF_REPO_ID, revision=level, token=HF_TOKEN)
    tokenizer.push_to_hub(HF_REPO_ID, revision=level, token=HF_TOKEN)
    
    del model, tokenizer, trainer, dataset
    gc.collect()
    torch.cuda.empty_cache()

print("\n\n🎉 레벨별 500개, 총 1,500개 데이터 학습 및 업로드가 완료되었습니다!")